In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import geopandas as gpd
import utils
from utils import util_functions, visualizer_func

In [13]:
df, twi_df = util_functions.get_processed_merged_data()

c:\src\jupyter_codes\foreign-affairs\utils\util_functions.py:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  country_list = df['focus group'][idx]


In [44]:
with open('findings/country_corps.json', 'r') as file:
    country_info_dict = json.load(file)

In [47]:
len(country_info_dict['Australia']['name'])

239

In [8]:
df['Instagram Follower #'].sum()
df['']

30186031.0

In [14]:
me = visualizer_func.plot_filtered_language_follower_bar(df)

2023-11-19 18:24:40.379 
  command:

    streamlit run c:\Users\sinaz\.conda\envs\test\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [ ]:
util_functions.create_country_to_name_owner_parent_data(df)

In [54]:
with open('findings/country_corps.json', 'r') as file:
    country_info_dict = json.load(file)
len(country_info_dict['Australia']['owner'])

32

In [ ]:
country_info = country_info_dict[selected_country_name]

# Ensure that all lists have the same length
min_length = min(map(len, country_info.values()))
country_info = {key: value[:min_length] for key, value in country_info.items()}

# Create a DataFrame from the country_info dictionary
country_df = pd.DataFrame(country_info)

# Display the scrollable table
st.dataframe(country_df.style.set_properties(**{'max-height': '400px', 'overflow-y': 'auto'}))

In [43]:
selected_language = 'English'
selected_column = 'X (Twitter) Follower #'

filtered_df = df[df['Language'] == selected_language]
filtered_df = filtered_df[filtered_df[selected_column].notna()]
filtered_df = filtered_df.set_index('Account Name')
filtered_df = filtered_df.sort_values(selected_column, ascending=False)
filtered_df.head(1)

,Region of Focus,Language,Entity owner (English),Parent entity (English),X (Twitter) handle,X (Twitter) URL,X (Twitter) Follower #,Facebook page,Facebook URL,Facebook Follower #,...,Name (English),focus group,is_blue_verified,created_at,followers_count,following_count,tweet_num,followers_following_ratio,formatted_created_at,type
Account Name,,,,,,,,,,,,,,,,,,,,,
CGTN,Anglosphere,English,China Media Group (CMG),Central Publicity Department,CGTNOfficial,https://twitter.com/CGTNOfficial,13000000.0,ChinaGlobalTVNetwork,https://www.facebook.com/ChinaGlobalTVNetwork,116450496.0,...,CGTN,"[Australia, Canada, New Zealand, United Kingdo...",True,2013-01-24 03:18:59,12997461.0,76.0,287284.0,171019.223662,2013-01-24,Media & News Company


In [33]:


# filtered_df[filtered_df[selected_column].notna()].set_index('Account Name', inplace=True)
# filtered_df.info()

,Region of Focus,Language,Entity owner (English),Parent entity (English),X (Twitter) handle,X (Twitter) URL,X (Twitter) Follower #,Facebook page,Facebook URL,Facebook Follower #,...,Name (English),focus group,is_blue_verified,created_at,followers_count,following_count,tweet_num,followers_following_ratio,formatted_created_at,type
Account Name,,,,,,,,,,,,,,,,,,,,,
CGTN,Anglosphere,English,China Media Group (CMG),Central Publicity Department,CGTNOfficial,https://twitter.com/CGTNOfficial,13000000.0,ChinaGlobalTVNetwork,https://www.facebook.com/ChinaGlobalTVNetwork,116450496.0,...,CGTN,"[Australia, Canada, New Zealand, United Kingdo...",True,2013-01-24 03:18:59,12997461.0,76.0,287284.0,171019.223662,2013-01-24,Media & News Company
China Daily,Anglosphere,English,China Daily Press,Central Publicity Department,ChinaDaily,https://twitter.com/ChinaDaily,4100000.0,chinadaily,https://www.facebook.com/chinadaily,104613175.0,...,China Daily,"[Australia, Canada, New Zealand, United Kingdo...",True,2009-11-05 20:30:10,4164585.0,157.0,211333.0,26526.019107,2009-11-05,Media & News Company
Hua Chunying,Anglosphere,English,Ministry of Foreign Affairs,Ministry of Foreign Affairs,SpokespersonCHN,https://twitter.com/SpokespersonCHN,2200000.0,profile.php?id=100092522612975,https://www.facebook.com/profile.php?id=100092...,744.0,...,Hua Chunying,"[Australia, Canada, New Zealand, United Kingdo...",True,2019-10-14 10:56:54,2321222.0,218.0,11310.0,10647.807339,2019-10-14,NaN
People's Daily China Science,Anglosphere,English,People's Daily Press,Central Committee of the Chinese Communist Party,ChinaScience,https://twitter.com/ChinaScience,2000000.0,PDChinaScience/,https://www.facebook.com/PDChinaScience/,5800000.0,...,People's Daily China Science,"[Australia, Canada, New Zealand, United Kingdo...",True,2019-08-23 16:40:11,2036766.0,10.0,10007.0,203676.599796,2019-08-23,NaN
Zhao Lijian,Anglosphere,English,Ministry of Foreign Affairs,Ministry of Foreign Affairs,zlj517,https://twitter.com/zlj517,1900000.0,NaN,NaN,NaN,...,Zhao Lijian,"[Australia, Canada, New Zealand, United Kingdo...",True,2010-05-08 15:38:36,1924909.0,169385.0,71656.0,11.364105,2010-05-08,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wang Tianyu (Evan Wang),Anglosphere,English,China Media Group (CMG),Central Publicity Department,wty510,https://twitter.com/wty510,7.0,NaN,NaN,NaN,...,Wang Tianyu (Evan Wang),"[Australia, Canada, New Zealand, United Kingdo...",False,2014-09-02 06:50:49,9.0,109.0,19.0,0.082569,2014-09-02,NaN
Zheng Chenlei (Blair Zheng),Anglosphere,English,China Media Group (CMG),Central Publicity Department,BlairZheng1,https://twitter.com/BlairZheng1,7.0,NaN,NaN,NaN,...,Zheng Chenlei (Blair Zheng),"[Australia, Canada, New Zealand, United Kingdo...",False,2012-11-07 05:04:27,8.0,12.0,27.0,0.666667,2012-11-07,NaN
Zhang Lin (Melody Zhang),Anglosphere,English,China Media Group (CMG),Central Publicity Department,MelodyZ77141898,https://twitter.com/MelodyZ77141898,3.0,profile.php?id=100060124341866,https://www.facebook.com/Money-Honey-with-Melo...,962000.0,...,Zhang Lin (Melody Zhang),"[Australia, Canada, New Zealand, United Kingdo...",False,2021-02-08 12:39:16,4.0,27.0,13.0,0.148148,2021-02-08,NaN


In [40]:
selected_column

'X (Twitter) Follower #'

In [23]:
selected_column_series = filtered_df[selected_column]
# selected_column_series = selected_column_series.dropna()
# selected_column_series.sort_values(ascending=False, inplace=True)

,Region of Focus,Language,Entity owner (English),Parent entity (English),X (Twitter) handle,X (Twitter) URL,X (Twitter) Follower #,Facebook page,Facebook URL,Facebook Follower #,...,focus group,Account Name,is_blue_verified,created_at,followers_count,following_count,tweet_num,followers_following_ratio,formatted_created_at,type
0,Anglosphere,English,China Media Group (CMG),Central Publicity Department,_bubblyabby_,https://twitter.com/_bubblyabby_,1678.0,itsAbby-103043374799622,https://www.facebook.com/itsAbby-103043374799622,1387432.0,...,"[Australia, Canada, New Zealand, United Kingdo...",Yang Xinmeng (Abby Yang),True,2014-07-10 02:05:08,2799.0,161.0,851.0,17.385093,2014-07-10,NaN
1,Anglosphere,English,China Media Group (CMG),Central Publicity Department,_cultureexpress,https://twitter.com/_cultureexpress,2488.0,NaN,NaN,NaN,...,"[Australia, Canada, New Zealand, United Kingdo...",CGTN Culture Express,False,2017-05-22 10:49:21,2483.0,152.0,1601.0,16.335526,2017-05-22,NaN
4,Anglosphere,English,People's Daily Press,Central Committee of the Chinese Communist Party,AlexYsalex17,https://twitter.com/AlexYsalex17,2821.0,NaN,NaN,NaN,...,"[Australia, Canada, New Zealand, United Kingdo...",Yang Sheng,False,2016-08-10 09:15:01,2839.0,78.0,379.0,36.397436,2016-08-10,NaN
10,EU,English,Ministry of Foreign Affairs,Ministry of Foreign Affairs,Amb_ZhuJing,https://twitter.com/Amb_ZhuJing,18500.0,NaN,NaN,NaN,...,"[Austria, Belgium, Bulgaria, Croatia, Cyprus, ...",Zhu Jing,False,2020-01-09 10:28:54,18853.0,542.0,2370.0,34.784133,2020-01-09,NaN
12,African Union,English,Ministry of Foreign Affairs,Ministry of Foreign Affairs,Ambassador_Liu,https://twitter.com/Ambassador_Liu,11200.0,NaN,NaN,NaN,...,"[Nigeria, Egypt, South Africa, Algeria, Morocco]",Liu Yuxi,False,2019-10-22 08:14:29,11237.0,502.0,9953.0,22.384462,2019-10-22,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749,Singapore,English,Ministry of Foreign Affairs,Ministry of Foreign Affairs,NaN,NaN,NaN,Chinaemb.SG,https://www.facebook.com/Chinaemb.SG,101000.0,...,[Singapore],Chinese Embassy in Singapore,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
750,South Africa,English,Ministry of Foreign Affairs,Ministry of Foreign Affairs,NaN,NaN,NaN,chinainsa,https://www.facebook.com/chinainsa,76000.0,...,[South Africa],Chinese Embassy in South Africa,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
751,Trinidad and Tobago,English,Ministry of Foreign Affairs,Ministry of Foreign Affairs,NaN,NaN,NaN,ChineseEmbinTT,https://www.facebook.com/ChineseEmbinTT,1100.0,...,[Trinidad and Tobago],Chinese Embassy in Trinidad and Tobago,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
753,Greece,English,Ministry of Foreign Affairs,Ministry of Foreign Affairs,NaN,NaN,NaN,profile.php?id=100076260574528,https://www.facebook.com/profile.php?id=100076...,1000.0,...,[Greece],Xiao Junzheng,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
selected_column_series

89     13000000.0
137     4100000.0
461     2200000.0
197     2000000.0
419     1900000.0
          ...    
507           7.0
52            7.0
398           3.0
551           3.0
75            0.0
Name: X (Twitter) Follower #, Length: 370, dtype: float64

In [19]:
selected_column_series

89     13000000.0
137     4100000.0
461     2200000.0
197     2000000.0
419     1900000.0
          ...    
507           7.0
52            7.0
398           3.0
551           3.0
75            0.0
Name: X (Twitter) Follower #, Length: 370, dtype: float64

In [11]:
def generate_visual_numbers(num):
    if int(num /1000000000) > 0:
        bil = num /1000000000
        print(f'{bil:.2f} B')
    else:
        mil = num/1000000
        print(f'{mil:.2f} M')
generate_visual_numbers(1047829628)
generate_visual_numbers(70344703)

1.05 B
70.34 M


In [3]:
util_functions.generate_corps_data(df)

In [4]:
corp_dict = util_functions.load_file('findings/corp_data.json')
corp_dict

{'name': 758,
 'owner': 87,
 'parent': 69,
 'twitter': 574,
 'twitter_fol': 70344703,
 'tiktok': 121,
 'tiktok_fol': 20230414,
 'instagram': 140,
 'instagram_fol': 14787228,
 'youtube': 158,
 'youtube_fol': 30186031,
 'facebook': 325,
 'facebook_fol': 1047829628,
 'total': 759}

## Rest

In [ ]:
df.columns

In [3]:
df.head(1)

,Region of Focus,Language,Entity owner (English),Parent entity (English),X (Twitter) handle,X (Twitter) URL,X (Twitter) Follower #,Facebook page,Facebook URL,Facebook Follower #,...,TikTok Subscriber #,Name (English),focus group,is_blue_verified,created_at,followers_count,following_count,tweet_num,followers_following_ratio,formatted_created_at
0,Anglosphere,English,China Media Group (CMG),Central Publicity Department,_bubblyabby_,https://twitter.com/_bubblyabby_,1678.0,itsAbby-103043374799622,https://www.facebook.com/itsAbby-103043374799622,1387432.0,...,660.0,Yang Xinmeng (Abby Yang),"[Australia, Canada, New Zealand, United Kingdo...",True,2014-07-10 02:05:08,2799.0,161.0,851.0,17.385093,2014-07-10


In [4]:
temp_series = df['is_blue_verified'].value_counts(dropna=False)
temp_df = pd.DataFrame({'Name': ["Is Verified", "Is Not Verified", "N/A"], 'Value': list(temp_series.values)})
temp_df

,Name,Value
0,Is Verified,443
1,Is Not Verified,204
2,N/A,111


In [5]:
list(temp_series.index)

[False, nan, True]

In [6]:
temp_df = pd.DataFrame({'Name': [], 'Value': []})